In [ ]:
# ~~~~~~Please use it at first time~~~~~~~~~~~~
!pip install numpy matplotlib xarray datetime tqdm pandas

In [ ]:
# Install additionall Packages (Need Internet Connection) (current Version no need to install)
!pip install git+https://github.com/openearth/dfm_tools.git

In [ ]:
# Please add this reprosity of the Delft3D to your directory 
# "https://svn.oss.deltares.nl/repos/openearthtools/trunk/python/OpenEarthTools/openearthtools"
# note that you should have account in (https://oss.deltares.nl/nl/web/delft3d)

In [ ]:
import sys
import numpy as np
import pandas as pd
import openearthtools.io.delft3d as d3d
import xarray as xr
import datetime
#from dask.distributed import Client
import warnings
from tqdm.notebook import tnrange

warnings.filterwarnings("ignore")
#client = Client(n_workers=4,memory_limit='16GB')
#client

# Delft3D-FLOW

In [ ]:
print('\nstart read .nc file\n')

# INPUT NETCDF4 FILE NAME (HYCOM)
nc_fileID = str(input('Input netcdf file name: '))

path = 'Delft3D-FLOW Folder'
G = d3d.Grid.fromfile(path +'*.grd')
D = pd.DataFrame(d3d.Dep.read(path +'*.dep',G.shape).val)
lyr = pd.read_csv(path +'*.lyr',sep='\s+',header=None).T
lyr.columns = ['depth']

df = pd.read_csv(path +'*.bnd',sep='\s+',header=None)
df.columns = ['name_boundary','type_boundary','type_data','A_x','A_y','B_x','B_y','alpha','name_A','name_B']

x_A = []
y_A = []
x_B = []
y_B = []


for k in range(len(df)):
    if 'East' in df['name_boundary'].loc[k]:
        ind_y_A = df['A_y'].loc[k]
        ind_x_A = df['A_x'].loc[k]-2
        ind_y_B = df['B_y'].loc[k]
        ind_x_B = df['B_x'].loc[k]-2
        
    elif 'North' in df['name_boundary'].loc[k]:
        ind_y_A = df['A_y'].loc[k]-2
        ind_x_A = df['A_x'].loc[k]
        ind_y_B = df['B_y'].loc[k]-2
        ind_x_B = df['B_x'].loc[k]
    else:
        ind_y_A = df['A_y'].loc[k]
        ind_x_A = df['A_x'].loc[k]
        ind_y_B = df['B_y'].loc[k]
        ind_x_B = df['B_x'].loc[k]
        
    x_A.append(G.__dict__['x'][ind_y_A][ind_x_A])
    y_A.append(G.__dict__['y'][ind_y_A][ind_x_A])
    x_B.append(G.__dict__['x'][ind_y_B][ind_x_B-1])
    y_B.append(G.__dict__['y'][ind_y_B][ind_x_B-1])
        
df['x_A'] = x_A
df['y_A'] = y_A
df['x_B'] = x_B
df['y_B'] = y_B

reference_year = '20190101'
t0 = datetime.datetime.strptime(reference_year,'%Y%m%d')
start_time = '20190101'
t1 = datetime.datetime.strptime(start_time,'%Y%m%d')


ds = xr.open_dataset(nc_fileID)
ds = ds.sel({'time':slice('2019-01-01T00:00:00.000000000','2021-01-01T00:00:00.000000000')})
#ds = ds.interpolate_na(dim=["lat",'lon','depth','time'], method="spline", fill_value="extrapolate")
# LIST OF AVIABLE OUTPUT DATA FOR DELFT3D METEO INPUT FILES
fmt='.bcc'

outfile_name = 'name_of_file'+fmt
outfile = open(outfile_name, 'w')


for j in tnrange(len(df)):
    # WRITE DATA IN FILE
    # Depths find in middle
    depths = (lyr*D.iloc[df['A_y'].loc[0],df['A_x'].loc[0]]/200).cumsum(axis=0)

    outfile.write('table-name'+'           '+r"'Boundary Section : "+str(j+1)+r"'")
    outfile.write('\n')
    outfile.write('contents'+'             '+r"'3d-profile'")
    outfile.write('\n')
    outfile.write('location'+'             '+r"'"+df['name_boundary'].loc[j]+r"             '")
    outfile.write('\n')
    outfile.write('time-function'+'        '+r"'non-equidistant'")
    outfile.write('\n')
    outfile.write('reference-time'+'       '+str(reference_year))
    outfile.write('\n')
    outfile.write('time-unit'+'            '+r"'minutes'")
    outfile.write('\n')
    outfile.write('interpolation'+'        '+r"'linear'")
    outfile.write('\n')
    outfile.write('parameter'+'            '+r"'time                '"+'  '+'unit'+' '+r"'[min]'")
    outfile.write('\n')
    for k in range(len(lyr)):
        outfile.write('parameter'+'            '+r"'Salinity             end A"+' layer '+str(k+1)+r"  '"+'     '+'unit'+' '+r"'[ppt]'")
        outfile.write('\n')
    for k in range(len(lyr)):    
        outfile.write('parameter'+'            '+r"'Salinity             end B"+' layer '+str(k+1)+r"  '"+'     '+'unit'+' '+r"'[ppt]'")
        outfile.write('\n')
    outfile.write('records-in-table'+'     '+str(len(ds.time)))
    outfile.write('\n')
    
    for t in tnrange(len(ds.time),desc='Salinity'):
        outfile.write(' '+'{:.8e}'.format((ds.time[t].values - np.datetime64('2019-01-01T00:00:00Z')) / np.timedelta64(1, 'm')))
        for k in range(len(lyr)):
            A = ds['salinity'].sel(lon=df['x_A'][j],lat = df['y_A'][j], depth = depths['depth'][k], method="nearest").isel(time=t).values
            outfile.write('  '+'{:.8e}'.format(np.nan_to_num(A)))
        for k in range(len(lyr)):
            B = ds['salinity'].sel(lon=df['x_B'][j],lat = df['y_B'][j], depth = depths['depth'][k], method="nearest").isel(time=t).values
            outfile.write('  '+'{:.8e}'.format(np.nan_to_num(B)))
    
        outfile.write('\n')

    outfile.write('table-name'+'           '+r"'Boundary Section : "+str(j+1)+r"'")
    outfile.write('\n')
    outfile.write('contents'+'             '+r"'3d-profile'")
    outfile.write('\n')
    outfile.write('location'+'             '+r"'"+df['name_boundary'].loc[j]+r"             '")
    outfile.write('\n')
    outfile.write('time-function'+'        '+r"'non-equidistant'")
    outfile.write('\n')
    outfile.write('reference-time'+'       '+str(reference_year))
    outfile.write('\n')
    outfile.write('time-unit'+'            '+r"'minutes'")
    outfile.write('\n')
    outfile.write('interpolation'+'        '+r"'linear'")
    outfile.write('\n')
    outfile.write('parameter'+'            '+r"'time                '"+'  '+'unit'+' '+r"'[min]'")
    outfile.write('\n')
    for k in range(len(lyr)):
        outfile.write('parameter'+'            '+r"'Temperature          end A"+' layer '+str(k+1)+r"  '"+'     '+'unit'+' '+r"'[°C]'")
        outfile.write('\n')
    for k in range(len(lyr)):    
        outfile.write('parameter'+'            '+r"'Temperature          end B"+' layer '+str(k+1)+r"  '"+'     '+'unit'+' '+r"'[°C]'")
        outfile.write('\n')
    outfile.write('records-in-table'+'     '+str(len(ds.time)))
    outfile.write('\n')
    for t in tnrange(len(ds.time),desc='Temperature'):
        outfile.write(' '+'{:.8e}'.format((ds.time[t].values - np.datetime64('2019-01-01T00:00:00Z')) / np.timedelta64(1, 'm')))
        for k in range(len(lyr)):
            A = ds['water_temp'].sel(lon=df['x_A'][j],lat = df['y_A'][j], depth = depths['depth'][k], method="nearest").isel(time=t).values
            outfile.write('  '+'{:.8e}'.format(np.nan_to_num(A)))
        for k in range(len(lyr)):
            B = ds['water_temp'].sel(lon=df['x_B'][j],lat = df['y_B'][j], depth = depths['depth'][k], method="nearest").isel(time=t).values
            outfile.write('  '+'{:.8e}'.format(np.nan_to_num(B)))
        outfile.write('\n')
    
#outfile.write('\n')
print('done')

# Delft3D-FM

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import io
fmt ='Salinity.bc'
path = '*.pli'

In [ ]:
df = pd.DataFrame(np.loadtxt(path,skiprows=2,dtype='str'))
df.columns = ['lon','lat','name']
ds = xr.open_dataset('HY.nc') #'HYCOM dataset'
layer = 5 #number of layers
depth_per = np.linspace(0,100,layer,dtype='int')
fmt='Salinity.bc'
outfile_name = 'path_to_folder'+fmt
outfile = open(outfile_name, 'w')
for i in tnrange(len(df)):
    dds = ds.sel(lon=df.loc[i]['lon'],lat = df.loc[i]['lat'], method="nearest").dropna(dim='depth')
    dds.salinity['depth_per'] = 100*(dds.salinity['depth']/dds.salinity['depth'].max())
    outfile.write('[forcing]')
    outfile.write('\n')
    outfile.write('Name'+'                            = '+df.loc[i]['name'])
    outfile.write('\n')
    outfile.write('Function'+'                        = '+'t3d')
    outfile.write('\n')
    outfile.write('Time-interpolation'+'              = '+'linear')
    outfile.write('\n')
    outfile.write('Vertical position type'+'          = '+'percentage from bed')
    outfile.write('\n')
    outfile.write('Vertical position specification'+' =')
    for ddsdp in depth_per:
        outfile.write(' '+str(int(ddsdp)))
    outfile.write('\n')
    outfile.write('Vertical interpolation'+'          = '+'linear')
    outfile.write('\n')
    outfile.write('Quantity'+'                        = '+'time')
    outfile.write('\n')
    outfile.write('Unit'+'                            = '+'seconds since '+'2018-01-01 00:00:00')
    outfile.write('\n')
    for ddsdp_int in range(1,len(depth_per)+1):
        outfile.write('Quantity'+'                        = '+'salinitybnd')
        outfile.write('\n')
        outfile.write('Unit'+'                            = '+'ppt')
        outfile.write('\n')
        outfile.write('Vertical position'+'               = '+str(ddsdp_int))
        outfile.write('\n')
    time=0
    for t in dds.time:
        outfile.write(str(int((t.values - np.datetime64('2018-01-01T00:00:00Z')) / np.timedelta64(1, 's'))))
        
        for ddsdp_int in depth_per:
            outfile.write('     '+str(np.round(dds.salinity.sel(depth=int((ddsdp_int/100)*float(dds.salinity.depth.max().values)),method='nearest').isel(time=time).values,decimals=3)))
        time+=1
        outfile.write('\n')

In [ ]:
df = pd.DataFrame(np.loadtxt(path,skiprows=2,dtype='str'))
df.columns = ['lon','lat','name']
ds = xr.open_dataset('HY.nc') #HYCOM dataset
layer = 5 #number of layers
fmt='Temperature.bc'
depth_per = np.linspace(0,100,layer,dtype='int')
outfile_name = 'path_to_folder'+fmt
outfile = open(outfile_name, 'w')
for i in tnrange(len(df)):
    dds = ds.sel(lon=df.loc[i]['lon'],lat = df.loc[i]['lat'], method="nearest").dropna(dim='depth')
    dds.water_temp['depth_per'] = 100*(dds.water_temp['depth']/dds.water_temp['depth'].max())
    outfile.write('[forcing]')
    outfile.write('\n')
    outfile.write('Name'+'                            = '+df.loc[i]['name'])
    outfile.write('\n')
    outfile.write('Function'+'                        = '+'t3d')
    outfile.write('\n')
    outfile.write('Time-interpolation'+'              = '+'linear')
    outfile.write('\n')
    outfile.write('Vertical position type'+'          = '+'percentage from bed')
    outfile.write('\n')
    outfile.write('Vertical position specification'+' =')
    for ddsdp in depth_per:
        outfile.write(' '+str(int(ddsdp)))
    outfile.write('\n')
    outfile.write('Vertical interpolation'+'          = '+'linear')
    outfile.write('\n')
    outfile.write('Quantity'+'                        = '+'time')
    outfile.write('\n')
    outfile.write('Unit'+'                            = '+'seconds since '+'2018-01-01 00:00:00')
    outfile.write('\n')
    for ddsdp_int in range(1,len(depth_per)+1):
        outfile.write('Quantity'+'                        = '+'temperaturebnd')
        outfile.write('\n')
        outfile.write('Unit'+'                            = '+'°C')
        outfile.write('\n')
        outfile.write('Vertical position'+'               = '+str(ddsdp_int))
        outfile.write('\n')
    time=0
    for t in dds.time:
        outfile.write(str(int((t.values - np.datetime64('2018-01-01T00:00:00Z')) / np.timedelta64(1, 's'))))
        
        for ddsdp_int in depth_per:
            outfile.write('     '+str(np.round(dds.water_temp.sel(depth=int((ddsdp_int/100)*float(dds.water_temp.depth.max().values)),method='nearest').isel(time=time).values,decimals=3)))
        time+=1
        outfile.write('\n')